# **Imports and setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
pip install scikit-dsp-comm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import numpy as np
import numpy.matlib
import tensorflow as tf
from scipy import signal
from scipy.signal import butter, lfilter, freqz
from scipy.signal import find_peaks
from scipy.signal import argrelextrema
import pandas as pd
import xlrd
import os
import librosa
import librosa.display
import math
import sys
import pandas as pd
import sk_dsp_comm.sigsys as ss
import sk_dsp_comm.fir_design_helper as fir_d
import openpyxl
from openpyxl import Workbook

# **data**

In [30]:
# read the data table:
%cd /content/drive/MyDrive/USVProject/running_data/

data_table = xlrd.open_workbook('Data 2015 For Syl Segmentation_1.xlsx').sheet_by_index(0)
mother = data_table.col_values(0, 1)
matgen = data_table.col_values(1, 1)
name = data_table.col_values(2, 1)
sex = data_table.col_values(3, 1)
pupgen = data_table.col_values(4, 1)
age = data_table.col_values(5, 1)
session = data_table.col_values(6, 1)
rec_num = data_table.col_values(7, 1)


/content/drive/.shortcut-targets-by-id/1zZ_ZmjBKjN3HmpYadLwvXXlHkCyi5dM9/USV_Project/running_data


In [31]:
#find path and load recordings:
SignalVec = []
signal_name = []
sr = 250000
for i in range(len(mother)):
  path = '/content/drive/MyDrive/USVRecordings/2015/{}_{}/{}_{}/day_{}/session{}/{}.wav'.format(mother[i], matgen[i], name[i], pupgen[i], int(age[i]), int(session[i]), rec_num[i]) #find path of each recording
  if not os.path.exists('{}'.format(path)):
    path = '/content/drive/MyDrive/USVRecordings/2015/{}_{}/{}_{}/day_{}/session{}/{}.WAV'.format(mother[i], matgen[i], name[i], pupgen[i], int(age[i]), int(session[i]), rec_num[i])
    if not os.path.exists('{}'.format(path)):
      print(i)
      continue
  signal_name.append(path)
  rec, rate = librosa.load(path, sr) #opens recordings and sample rate
  SignalVec.append(rec)

141
142
144
149
152
153
154
155
156
158
163
164
165
170
171
175
178
185
189
195
198
203
204
207
208
209
210
211
212
214
216
218
219
221
224
232
235
241
242
244
248
249
251
253
255
256
258
260
261
263
264
266
267
269
271
272
273
275
276
279
281
282
283
284
290
293
295
299
300
301
303
328
338
350
354
355
358
376
377
389
395
405
406
407
411
415
419
423
424
427
428
430
453
468
470
517
520
522
524
527
529
533
534
535
542
545
547
573
595
596
597
600
601
602
643
648
654
655
666
670
675


# **Functions**

'Preprocessing' function:

In [32]:
def Preprocessing(signal,Fs):
  signal = signal-np.mean(signal) #remove mean

  b_r = fir_d.fir_remez_lpf(100000,120000,0.5,40,250000) #lowpass FIR filter
  signal1 = lfilter(b_r, [1], signal)

  b_rF = fir_d.fir_remez_bsf(48000,49500,50500,52000,0.5,40,250000) #filter noise in 50000 Hz 
  signal2 = lfilter(b_rF, [1], signal1)
  return signal2

'framing' function:

In [ ]:
# output- newSignalFrames - The new signal's Frames.
#         mind - The time in samples at the centre of each frame.
def farming(signal,Fs,FrameLength,Overlap):
  win = int(FrameLength*Fs); 
  inc = round((Overlap)*win);
  signalMAT = tf.signal.frame(signal,win,inc)
  nx = len(signal) #the signal length
  nf = math.floor((nx-win+inc)/inc) #the number of segmants
  indf = inc*np.array(range(0,nf)) #the initial index of the segment
  mind = (1+win)/2+indf #the time in samples at the centre of each frame
  w = np.hamming(win)
  W = np.matlib.repmat(w, nf, 1)
  newSignalFrames = signalMAT*W
  return newSignalFrames, mind

'MakeERBFilters' function:

In [ ]:
# MakeERBFilters function computes the filter coefficients for a bank of Gammatone filters.
# These filters were defined by Patterson and Holdworth for simulating the cochlea
# The results are returned as arrays of filter coefficients. 

def MakeERBFilters(Fs,numChannels,lowFreq):
  T=1/Fs; # half the sampling rate (Fs)
  EarQ = 9.26449; # Glasberg and Moore Parameters
  minBW = 24.7;
  order = 1;
  cf = -(EarQ*minBW)+np.exp(np.atleast_2d(np.array(range(1,numChannels+1))).T*(-math.log(Fs/2 + EarQ*minBW) + math.log(lowFreq + EarQ*minBW))/numChannels)*(Fs/2 + EarQ*minBW);
  ERB = pow((pow((cf/EarQ),order) + pow(minBW,order)),(1/order))
  B=1.019*2*math.pi*ERB
  gain = abs((-2*np.exp(4j*cf*math.pi*T)*T + 2*np.exp(-(B*T) + 2j*cf*math.pi*T)*T*(np.cos(2*cf*math.pi*T) - np.sqrt(3 - pow(2,(3/2)))*np.sin(2*cf*math.pi*T)))*(-2*np.exp(4j*cf*math.pi*T)*T + 2*np.exp(-(B*T) + 2j*cf*math.pi*T)*T*(np.cos(2*cf*math.pi*T) + np.sqrt(3 - pow(2,(3/2)))*np.sin(2*cf*math.pi*T)))*(-2*np.exp(4j*cf*math.pi*T)*T + 2*np.exp(-(B*T) + 2j*cf*math.pi*T)*T*(np.cos(2*cf*math.pi*T) - np.sqrt(3 + pow(2,(3/2)))*np.sin(2*cf*math.pi*T)))*(-2*np.exp(4j*cf*math.pi*T)*T + 2*np.exp(-(B*T) + 2j*cf*math.pi*T)*T*(np.cos(2*cf*math.pi*T) + np.sqrt(3 + pow(2,(3/2)))*np.sin(2*cf*math.pi*T)))/pow((-2/np.exp(2*B*T) - 2*np.exp(4j*cf*math.pi*T) + 2*(1 + np.exp(4j*cf*math.pi*T))/np.exp(B*T)),4))

  forward = np.array([pow(T,4)/gain,-4*pow(T,4)*np.cos(2*cf*math.pi*T)/np.exp(B*T)/gain,6*pow(T,4)*np.cos(4*cf*math.pi*T)/np.exp(2*B*T)/gain,-4*pow(T,4)*np.cos(6*cf*math.pi*T)/np.exp(3*B*T)/gain,pow(T,4)*np.cos(8*cf*math.pi*T)/np.exp(4*B*T)/gain])
  forward = forward.transpose()
  forward = np.reshape(forward, (len(cf),5))
  feedback = np.array([np.ones((len(cf), 1), dtype=int),-8*np.cos(2*cf*math.pi*T)/np.exp(B*T),4*(4 + 3*np.cos(4*cf*math.pi*T))/np.exp(2*B*T),-8*(6*np.cos(2*cf*math.pi*T) + np.cos(6*cf*math.pi*T))/np.exp(3*B*T),2*(18 + 16*np.cos(4*cf*math.pi*T) + np.cos(8*cf*math.pi*T))/np.exp(4*B*T),-8*(6*np.cos(2*cf*math.pi*T) + np.cos(6*cf*math.pi*T))/np.exp(5*B*T),4*(4 + 3*np.cos(4*cf*math.pi*T))/np.exp(6*B*T),-8*np.cos(2*cf*math.pi*T)/np.exp(7*B*T),np.exp(-8*B*T)])
  feedback = feedback.transpose()
  feedback = np.reshape(feedback, (len(cf),9))
  return forward, feedback, cf

'ERBFilterBan' function:

In [ ]:
def ERBFilterBank(forward,feedback,x):
  # This function filters the waveform x with the array of filters specified by the forward and feedback parameters. 
  # Each row of the forward and feedback parameters are the parameters to the Matlab builtin function "filter".
  rows, cols = np.shape(feedback)
  y = []
  for i in range(rows):
    fil = lfilter(forward[i],feedback[i],x)
    y = np.append(y, fil)
  y = np.reshape(y, (rows, int(FrameLength*Fs)))
  return y

'Syllables_Detection_ERB' function:

In [ ]:
def Syllables_Detection_ERB(signal,ERBforward,ERBfeedback, f, silence_th, th, *M):
  # This function detects frequency components of input syllables
  f = f.transpose()
  peaks =[]
  F = np.zeros((1,3),dtype=int)
  mean_Full_Energy = np.mean(pow(signal,2))
  E = ERBFilterBank(ERBforward,ERBfeedback,signal).T # filter the signal through the filter bank, which each row is a result from one filter
  E2 = pow(E,2)
  Energy = E2.sum(axis=0) #total energy after filtering the frames
  Max = np.amax(Energy)
  we = np.where(Energy>(0.2*Max))
  Freq_var = len(we[0])
  if len(sys.argv)-1 == 7:
    silence_th = silence_th - 5
  if Freq_var <= silence_th: #if it's 1 than we've got vocal frame
    ind,_ = find_peaks(Energy)
    p = Energy[ind]
    thresh = th*Max
    for i in range(len(p)):
      if p[i]>=thresh:
        peaks.append(ind[i])
    if len(sys.argv)-1 == 7: #if looking for harmony:          
      thresh = th*M #adjust threshold
      for i in range(len(p)):
        if  (p[i]>=thresh) and (p[i]>(10*mean_Full_Energy)):
         peaks.append(ind[i])
      is_empty = p.size == 0
      if (is_empty):
        peaks = np.where((Energy>thresh) and (Energy>(100*mean_Full_Energy))) #look for any 
    is_empty = len(peaks) == 0
    if not(is_empty): #organize frequencies     
      I = np.where(np.diff(peaks)<2)
      is_empty = len(I) == 0
      if not(is_empty):
        for i in range(len(I)):
          peaks = np.delete(peaks, I[i]+1)
      if len(peaks)>=3:
        F = [[f[0,peaks[0]], f[0,peaks[1]], f[0,peaks[2]]]]
      else:
        if len(peaks)==2:
          F = [[f[0,peaks[0]], f[0,peaks[1]], 0]]
        else: 
          F = [[f[0,peaks[0]], 0, 0]] 
  return F, Max

'frequency_continuity' function:

In [ ]:
# This function forces continuity of harmonics in syllables The condition for 
# Harmony is at least 4 time frames where a harmony was found.
def frequency_continuity(x):
  i=0
  while i<=(len(x)-3):
    if x[int(i)][1]:
      if x[i+1,1] and x[i+2,1]:
        n = np.where(np.logical_not(x[range(i,len(x)),1]))
        next = n[0] + i
        is_empty = len(next) == 0
        if is_empty:
          break
        i = next[0]-1
      else: 
        x[i,2] = x[i,1]
        x[i,1] = 0
    i = i+1
    
  # last 3 lines
  if not(all(x[range(len(x)-3,-1),1])):
    x[range(i,-1),2] = x[range(i,-1),1]
    x[range(i,-1),1] = 0
  K = 1
  x = sorted(x, key = lambda ele: ele[K], reverse = True)
  return x                   

'Syllables_Detection2' function:

In [ ]:
def Syllables_Detection2(signal,Fs,FrameLength,Overlap, thresh, harmony_th, signal_name):
  Frames, mind = farming(signal,Fs,FrameLength,Overlap) #call framing function 

  SyllableVec = np.zeros((len(Frames),1),dtype=int)
  Formants = np.zeros((len(Frames),1),dtype=int)
  AllFormant = np.zeros((len(Frames),3),dtype=int)

  start_time = (np.array(ind2[1]))/Fs
  Time = start_time+mind/Fs
  StartEndTime = [Time-FrameLength/2,Time+FrameLength/2] #time at center of each frame

  numChannels = 90
  lowFreq = 35000
  ERBforward, ERBfeedback, f = MakeERBFilters(Fs,numChannels,lowFreq); #call MakeERBFilters function

  for frm in range(len(Frames)):
    s = Frames[frm]
    if all(v == 0 for v in s):
      continue
    
    th = 0.9 # first frequency energy threshold
    silence_th = thresh # first frequency silence threshold
    F, M = Syllables_Detection_ERB(s,ERBforward,ERBfeedback, f, silence_th, th)
    if any(F[0]):
      SyllableVec[frm] = 1
      nz = np.nonzero(F[0])
      if len(nz[0]) == 1:
         th_2 = harmony_th
         lowF = F[0][int(nz[0])]
         Filter_num = math.ceil((max(125000, lowF*2)-min(120000, lowF*2-2000))/(1.7*10000))
         if Filter_num<3: #because can't find peaks if shorter than 3
           Filter_num = 3
         ERBforward_2,ERBfeedback_2, f_2 = MakeERBFilters(250000,Filter_num,min(120000, lowF*2-2000))
         F_2,_ = Syllables_Detection_ERB(s,ERBforward_2,ERBfeedback_2, f_2, silence_th, th_2, M)
         if any(F_2[0]):
           Fnew = [F_2[0][0], F[0][0], F[0][1]]
           F[0] = sorted(Fnew, reverse=True)
           F_2.clear()
           Fnew.clear()
      AllFormant[frm] = F[0]
      F = []
  if not(any(SyllableVec)):
    SybTime = [] 
    SybForm = []
    ClassLPC = []
    return SybTime, SybForm, Formants, Time, ClassLPC, SyllableVec, Frames    
  else:
    Ind = np.where(SyllableVec.T)
    diff_ind = np.diff(np.insert(Ind[1], 0, 0))
    diff_ind[0] = diff_ind[0]+1
    skipedframe = []
    for i in range(len(diff_ind)):
      if diff_ind[i]>1 and diff_ind[i]<=4:
        skipedframe.append(Ind[1][i])
    is_empty = len(skipedframe) == 0
    if not(is_empty):
      for skip in range(len(skipedframe)):
        for se in range(max(1,skipedframe[skip]-3),skipedframe[skip]):
          SyllableVec[se] = 1 
        # fix allformants in skipped frames
        nonZ = np.nonzero(AllFormant[range(0, skipedframe[skip]),0])
        is_empty = len(nonZ[0]) == 0
        if (is_empty):
          before = 0
        else:
          before = np.max(nonZ[0])
        after = skipedframe[skip]
        for i in np.linspace(before+1, after-1, num=(after-1)-(before+1)+1):
          AllFormant[int(i),0] = np.mean([AllFormant[before,0], AllFormant[after,0]])
    Ind = np.where(SyllableVec.T)
    SybTime = Time[Ind[1]]
    SybForm = Formants[Ind[1]]
    AllFormant = frequency_continuity(AllFormant)
    class ClassLPC:
      time1 = [StartEndTime[0][Ind[1]], StartEndTime[1][Ind[1]]]
      formant = []
      for i in range(len(Ind[1])):
        formant.append(AllFormant[i]) # length of time X 3 (the 3 major formant detected in the time frame)
      fileName = signal_name
      window = FrameLength
      overlap = Overlap
  return SybTime, SybForm, Formants, Time, ClassLPC, SyllableVec, Frames    
  

'Rearrange_signal' function:

In [ ]:
def Rearrange_signal(signal,Fs,StartEnd):
  Between =[StartEnd[0][1:len(StartEnd[0])]-StartEnd[1][0:-1]]
  Between1 = np.insert(Between[0], 0, 1)
  bz1 = [Between1>0]
  Between2 = np.insert(Between[0], len(Between[0]), 1)
  bz2 = [Between2>0]

  StartEndNew = [StartEnd[0][bz1], StartEnd[1][bz2]]
  return StartEndNew

'Check_length_Call' function:

In [ ]:
def Check_length_Call(Start_End):
  # Fix 2 Time Limits
  # 1- proximity of two detections-> 1 Call!!
  # 2- Normal length of Call

  MIN_LENGTH = 0.01
  MAX_LENGTH = 0.3
  MIN_BETWEEN_CALL = 0.02
  is_empty = len(Start_End[0]) == 0
  if is_empty:  #Input Check
    New_Start_End = []
    return New_Start_End
  
  if len(Start_End[0])>1:  # in case the length is over 1 
    between = [Start_End[0][1:len(Start_End[0])]-Start_End[1][0:-1]]
    combineCalls = np.where(between[0]<MIN_BETWEEN_CALL) #Indexs of less than minimum
    
    count = 0
    ii = 0
    tmp1 = []
    tmp2 = []
    while ii<=len(Start_End[0])-1:
      tmp1.append(Start_End[0][ii])
      tmpIdx = ii
      OK = np.in1d(ii,combineCalls[0])
      if OK:
        while np.in1d(ii,combineCalls[0]):
          ii = ii + 1
        tmp2.append(Start_End[1][ii])
      else:
        tmp2.append(Start_End[1][ii])
        count = count + 1
      ii = ii + 1
  else:
    tmp1 = Start_End[0]
    tmp2 = Start_End[1]
  
  duration = []
  Idx = []
  New_Start_End = []
  for i in range(len(tmp1)):
    duration.append(tmp2[i] - tmp1[i])
  for i in range(len(duration)):
    if duration[i]>=MIN_LENGTH:
      Idx.append(i)
  for i in range(len(Idx)):
    New_Start_End.append([tmp1[Idx[i]], tmp2[Idx[i]]])
  return New_Start_End


## **Main**

In [36]:
Fs = rate
FrameLength = 0.006
Overlap = 0.7
thresh = 20
harmony_th = 0.009

siz = len(SignalVec)
book = Workbook()
sheet = book.active
title = ['Path','Mother','Mother Genotype','Name','Sex','Offspring Genotype','Day','Session','Recording number','Start point(s)','End point(s)']
sheet.append(title)
for s2 in range(siz):
  signal = SignalVec[s2]
  signal = Preprocessing(signal,Fs)
  # if there is a 'silent' start (zeros), skipping to the "real" start:
  ind = np.where(signal == 0)
  is_empty = ind[0].size == 0
  if not(is_empty) and ind[0][0] == 0:
   DiffInd = np.diff(np.diff(ind))
   ind2 = np.where(DiffInd != 0)
   is_empty = ind2[0].size == 0
   if not(is_empty):
     for i in range(0,len(signal)-int(ind2[0])):
       signal[i] = signal[i+int(ind2[0])]
     i = range(len(signal)-int(ind2[0]),len(signal))
     signal = np.delete(signal,i)
   else:
     ind2 = [[0],[0]]
  else:
    ind2 = [[0],[0]]

  _,_,_,_,ClassLPC,SyllabelVec,SignalPath = Syllables_Detection2(signal,Fs,FrameLength,Overlap, thresh, harmony_th, signal_name[s2])

  if any(SyllabelVec):
    StartEndNew = Rearrange_signal(signal,Fs,ClassLPC.time1) #StartEndNew - times vector
    StEndMatF = Check_length_Call(StartEndNew)
    print(StEndMatF)

    #!/content/drive/MyDrive/final_project/segmentation
#    for i in range(len(StEndMatF)):
#      new_row = [signal_name[s2],mother[s2],matgen[s2],name[s2],sex[s2],pupgen[s2],age[s2],session[s2],rec_num[s2],StEndMatF[i][0],StEndMatF[i][1]]
#      sheet.append(new_row)
#book.save('New Segmentatio Data For Final Class_3.xlsx')


   



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


[]
[]
[[0.529202, 0.543602]]
[[0.512402, 0.552002], [0.642602, 0.661202]]
[[0.49980199999999997, 0.526802]]
[[0.483002, 0.615002], [0.680402, 0.753602], [0.856802, 0.900602]]
[[0.516602, 0.564602], [0.630002, 0.715802], [0.798002, 0.879602], [0.957602, 1.039202], [1.1298020000000002, 1.152602], [1.2936020000000001, 1.341602]]
[[0.504002, 0.543602], [0.638402, 0.699002], [0.781202, 0.837602], [0.924002, 1.0140019999999998], [1.104602, 1.1694019999999998], [1.268402, 1.287002]]
[[0.508202, 0.535202]]
[[0.504002, 0.577202], [0.739202, 0.829202], [0.898802, 0.988802], [1.058402, 1.1232019999999998]]
[[0.49980199999999997, 0.560402]]
[]
[[0.520802, 0.547802]]


KeyboardInterrupt: ignored

In [ ]:
signal = SignalVec[430]
signal = Preprocessing(signal,Fs)
  # if there is a 'silent' start (zeros), skipping to the "real" start:
ind = np.where(signal == 0)
is_empty = ind[0].size == 0
if not(is_empty) and ind[0][0] == 0:
  DiffInd = np.diff(np.diff(ind))
  ind2 = np.where(DiffInd != 0)
  is_empty = ind2[0].size == 0
  if not(is_empty):
    for i in range(0,len(signal)-int(ind2[0])):
      signal[i] = signal[i+int(ind2[0])]
    i = range(len(signal)-int(ind2[0]),len(signal))
    signal = np.delete(signal,i)
  else:
    ind2 = [[0],[0]]
else:
  ind2 = [[0],[0]]

_,_,_,_,ClassLPC,SyllabelVec,SignalPath = Syllables_Detection2(signal,Fs,FrameLength,Overlap, thresh, harmony_th, signal_name[s2])

if any(SyllabelVec):
    StartEndNew = Rearrange_signal(signal,Fs,ClassLPC.time1) #StartEndNew - times vector
    StEndMatF = Check_length_Call(StartEndNew)
    print(StEndMatF)